In [24]:
class PUR:
  def __init__(self, quantile_low=0.10, quantile_high=0.90,Start_date='2013-12-27',initial_capital=100000,max_buy =1000000, max_sell=1000000, gas = 0.001, URL ='https://raw.githubusercontent.com/SciEcon/bitcoin_golden_litecoin_silver/main/PU%20ratio%20%26%20Trading/BTC/BTC_2022_05_31.csv'):
    self.low = quantile_low
    self.high = quantile_high
    self.start = Start_date
    self.initial_capital = initial_capital
    self.max_buy = max_buy
    self.max_sell = max_sell
    self.gas = gas
    self.URL = URL
  def portfolio(self):
    import pandas as pd 
    import numpy as np
    pd.options.display.float_format = "{:,.2f}".format
    pd.set_option('display.max_columns', None)
    pd.set_option('display.max_rows', None)
    data=pd.read_csv(self.URL,index_col='Unnamed: 0')
    data.index=pd.to_datetime(data.index)
    data = data[data.index>=self.start]
    prices = data['PriceUSD']
    quantiles_low=[]
    quantiles_high=[]
    cashes = []
    bitcoins = []
    holdings = []
    states = []
    cash = self.initial_capital
    bitcoin = 0
    holding = 0
    state = "" 
    def buy(i, cash, bitcoin, price):
      units = cash/(price*(1+self.gas))
      if units < 1:
        buy_units = 0
      else:
        if units>self.max_buy:
          buy_units = self.max_buy
        else:
          buy_units = units
        print('index %d: buy %f units at price %.3f, current cash %.3f, current bitcoin %.3f' % (i, buy_units, price, cash, bitcoin))
      cost = buy_units*price*(1+self.gas)
      cash -= cost
      bitcoin += buy_units
      holding = bitcoin*price
      return buy_units, cash, bitcoin, holding
    def sell(i, cash, bitcoin, price):
      if bitcoin == 0:
        sell_units = 0
      else:
        if bitcoin > self.max_sell:
          sell_units = self.max_sell
        else:
          sell_units = bitcoin
        print('index %d: sell %.3f units at price %.3f, current cash %.3f, current bitcoin %.3f' % (i, sell_units, price, cash, bitcoin))
      bitcoin -=sell_units
      revenue = sell_units*price*(1-self.gas)
      cash += revenue
      holding = bitcoin*price
      return sell_units, cash, bitcoin, holding 

    for i in range(0,data.shape[0]):
      quantile_low = data['PU Ratio'].iloc[0:i].quantile(self.low)
      quantile_high = data['PU Ratio'].iloc[0:i].quantile(self.high)
      PUR = data['PU Ratio'].iloc[i]
      price = prices[i]
      if PUR <= quantile_low:
        buy_units, cash, bitcoin, holding = buy(i, cash, bitcoin, price)
        if buy_units>0:
          state = 1
        else:
          state = 0  
      elif PUR >= quantile_high:
        sell_units, cash, bitcoin, holding = sell(i, cash, bitcoin, price)
        if sell_units>0:
          state = -1
        else:
          state = 0
      else:
        state = 0
        bitcoin = bitcoin
        cash = cash
        holding =bitcoin*price ### holding updates when prices update even when there is no buy or sell actions

      states.append(state)
      cashes.append(cash)
      bitcoins.append(bitcoin)
      holdings.append(holding)
      quantiles_low.append(quantile_low)
      quantiles_high.append(quantile_high)

    data.loc[:,'state']=states
    data.loc[:,'cash']=cashes
    data.loc[:,'bitcoin']=bitcoins
    data.loc[:,'holding']=holdings
    data.loc[:,'quantiles_low']=quantiles_low
    data.loc[:,'quantiles_high']=quantiles_high
    data['total']=data['cash']+data['holding']
    data['roi_gross'] = (data['total']-self.initial_capital)/self.initial_capital*100
    data['roi_daily'] = data['total'].pct_change()
    data=data.fillna(0)
    data['sharpe'] = 0
    for i in range(0, data.shape[0]):
      if data['roi_daily'].iloc[0:i].std()==0:
        data['sharpe'].iloc[i]=0
      else:
        data['sharpe'].iloc[i]=(i**0.5)*np.mean(data['roi_daily'].iloc[0:i])/data['roi_daily'].iloc[0:i].std()
    data=data.fillna(0)
    return data
  def figure_signal(self,data):
    import plotly.graph_objects as go
    from plotly.subplots import make_subplots
    fig = make_subplots(specs=[[{"secondary_y": True}]])
    fig.add_trace(
        
        go.Scatter(
            mode='lines',
            name ='PU Ratio',
            x=data.index,
            y=data['PU Ratio'],
            line_color="blue",
            line_width=1,

        ),
        secondary_y=False,
    )
    fig.add_trace(
        
        go.Scatter(
            mode='lines',
            name='Price in USD',
            x=data.index,
            y=data['PriceUSD'],
            line_color="black",
            line_width=1
        ),
        secondary_y=True,
    )
    fig.add_trace(
    
    go.Scatter(
        mode='lines',
        name='PU Ratio ' +str(self.low)+ ' Quantile',
        x=data.index,
        y=data.quantiles_low,
        line_color="green",
        line_width=1,
    ),
    secondary_y=False,
    )
    fig.add_trace(
        
        go.Scatter(
            mode='lines',
            name='PU Ratio ' +str(self.high)+' Quantile',
            x=data.index,
            y=data.quantiles_high,
            line_color="red",
            line_width=1,
        ),
        secondary_y=False,
    )

    fig.add_trace(
        go.Scatter(name="buy",mode="markers", x=data.index[data.state==1], y=data['PriceUSD'][data.state==1], marker_symbol=119,
                              marker_line_color="green", marker_color="green",
                              marker_line_width=2, marker_size=15,
    ),
    secondary_y=True,
    )
    fig.add_trace(
        go.Scatter(name="sell",mode="markers", x=data.index[data.state==-1], y=data['PriceUSD'][data.state==-1], marker_symbol=120,
                              marker_line_color="red", marker_color="red",
                              marker_line_width=2, marker_size=15,
    ),
        secondary_y=True,
    )

    # Set y-axes titles
    fig.update_yaxes(title_text="<b>PU Ratio</b>", secondary_y=False)
    fig.update_yaxes(title_text="<b>Price in USD</b>", secondary_y=True)
    # Set y-axes title
    fig.update_xaxes(title_text="<b>date<b>")
    # set title 
    fig.update_layout(
        title_text="Buy and Sell Signals: PU Ratio"
    )
    # update templates
    fig.update_layout(template="plotly_white")
    fig.show()
    return
  def figure_portfolio(self,data):
    import plotly.graph_objects as go
    from plotly.subplots import make_subplots
    fig = make_subplots(specs=[[{"secondary_y": True}]])
    fig.add_trace(
        
        go.Scatter(
            mode='lines',
            name ='Cash',
            x=data.index,
            y=data['cash'],
            line_color="red",
            line_width=1,

        ),
        secondary_y=False,
    )
    fig.add_trace(
        
        go.Scatter(
            mode='lines',
            name='Holding',
            x=data.index,
            y=data['holding'],
            line_color="green",
            line_width=1
        ),
        secondary_y=False,
    )
    fig.add_trace(
        
        go.Scatter(
            mode='lines',
            name='Total',
            x=data.index,
            y=data['total'],
            line_color="blue",
            line_width=1
        ),
        secondary_y=False,
    )
    # Set y-axes titles
    fig.update_yaxes(title_text="<b>USD</b>", secondary_y=False)
    # Set y-axes title
    fig.update_xaxes(title_text="<b>date<b>")
    # set title 
    fig.update_layout(
        title_text="Portfolio Time Series: PU Ratio Automated Trading Strategies"
    )
    # update templates
    fig.update_layout(template="plotly_white")
    fig.show()
    return 
  def figure_returns(self,data):
    import plotly.graph_objects as go
    from plotly.subplots import make_subplots
    fig = make_subplots(specs=[[{"secondary_y": True}]])
    fig.add_trace(
          
        go.Scatter(
              mode='lines',
              name ='Gross ROI',
              x=data.index,
              y=data['roi_gross'],
              line_color="blue",
              line_width=1,

          ),
        secondary_y=False,
      )
    fig.add_trace(
          
        go.Scatter(
              mode='lines',
              name='Annualized Sharpe Ratio',
              x=data.index,
              y=data['sharpe'],
              line_color="black",
              line_width=1
          ),
        secondary_y=True,
      )
    # Set y-axes titles
    fig.update_yaxes(title_text="<b>Percent</b>", secondary_y=False)
    fig.update_yaxes(title_text="<b>Ratio</b>", secondary_y=True)
    # Set y-axes title
    fig.update_xaxes(title_text="<b>date<b>")
    # set title 
    fig.update_layout(
        title_text="Gross ROI and Annualized Sharpe Ratio: PU Ratio Automated Trading Strategy"
    )
    # update templates
    fig.update_layout(template="plotly_white")
    fig.show()
    return


In [25]:
PUR= PUR(quantile_low=0.10,quantile_high=0.90,max_buy=100,max_sell=100)

In [26]:
data= PUR.portfolio()

<ipython-input-26-0f609fd19f6d>:1: DtypeWarning:

Columns (146) have mixed types.Specify dtype option on import or set low_memory=False.



index 1: buy 100.000000 units at price 715.852, current cash 100000.000, current bitcoin 0.000
index 3: sell 100.000 units at price 735.742, current cash 28343.178, current bitcoin 100.000
index 21: buy 100.000000 units at price 792.830, current cash 101843.804, current bitcoin 0.000
index 22: buy 27.734710 units at price 809.784, current cash 22481.570, current bitcoin 100.000
index 185: sell 100.000 units at price 638.946, current cash 0.000, current bitcoin 127.735
index 186: sell 27.735 units at price 642.957, current cash 63830.717, current bitcoin 27.735
index 280: buy 100.000000 units at price 357.666, current cash 81645.099, current bitcoin 0.000
index 281: buy 100.000000 units at price 328.826, current cash 45842.761, current bitcoin 100.000
index 282: buy 40.004035 units at price 322.826, current cash 12927.267, current bitcoin 200.000
index 1343: sell 100.000 units at price 4740.356, current cash 0.000, current bitcoin 240.004
index 1344: sell 100.000 units at price 4918.166

/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [27]:
data.tail()

,AdrActCnt,AdrBal1in100KCnt,AdrBal1in100MCnt,AdrBal1in10BCnt,AdrBal1in10KCnt,AdrBal1in10MCnt,AdrBal1in1BCnt,AdrBal1in1KCnt,AdrBal1in1MCnt,AdrBalCnt,AdrBalNtv0.001Cnt,AdrBalNtv0.01Cnt,AdrBalNtv0.1Cnt,AdrBalNtv100Cnt,AdrBalNtv100KCnt,AdrBalNtv10Cnt,AdrBalNtv10KCnt,AdrBalNtv1Cnt,AdrBalNtv1KCnt,AdrBalNtv1MCnt,AdrBalUSD100Cnt,AdrBalUSD100KCnt,AdrBalUSD10Cnt,AdrBalUSD10KCnt,AdrBalUSD10MCnt,AdrBalUSD1Cnt,AdrBalUSD1KCnt,AdrBalUSD1MCnt,AssetEODCompletionTime,BlkCnt,BlkSizeMeanByte,BlkWghtMean,BlkWghtTot,CapAct1yrUSD,CapMVRVCur,CapMVRVFF,CapMrktCurUSD,CapMrktEstUSD,CapMrktFFUSD,CapRealUSD,DiffLast,DiffMean,FeeByteMeanNtv,FeeMeanNtv,FeeMeanUSD,FeeMedNtv,FeeMedUSD,FeeTotNtv,FeeTotUSD,FlowInExNtv,FlowInExUSD,FlowOutExNtv,FlowOutExUSD,FlowTfrFromExCnt,HashRate,HashRate30d,IssContNtv,IssContPctAnn,IssContPctDay,IssContUSD,IssTotNtv,IssTotUSD,NDF,NVTAdj,NVTAdj90,NVTAdjFF,NVTAdjFF90,PriceBTC,PriceUSD,ROI1yr,ROI30d,ReferenceRate,ReferenceRateETH,ReferenceRateEUR,ReferenceRateUSD,RevAllTimeUSD,RevHashNtv,RevHashRateNtv,RevHashRateUSD,RevHashUSD,RevNtv,RevUSD,SER,SplyAct10yr,SplyAct180d,SplyAct1d,SplyAct1yr,SplyAct2yr,SplyAct30d,SplyAct3yr,SplyAct4yr,SplyAct5yr,SplyAct7d,SplyAct90d,SplyActEver,SplyActPct1yr,SplyAdrBal1in100K,SplyAdrBal1in100M,SplyAdrBal1in10B,SplyAdrBal1in10K,SplyAdrBal1in10M,SplyAdrBal1in1B,SplyAdrBal1in1K,SplyAdrBal1in1M,SplyAdrBalNtv0.001,SplyAdrBalNtv0.01,SplyAdrBalNtv0.1,SplyAdrBalNtv1,SplyAdrBalNtv10,SplyAdrBalNtv100,SplyAdrBalNtv100K,SplyAdrBalNtv10K,SplyAdrBalNtv1K,SplyAdrBalNtv1M,SplyAdrBalUSD1,SplyAdrBalUSD10,SplyAdrBalUSD100,SplyAdrBalUSD100K,SplyAdrBalUSD10K,SplyAdrBalUSD10M,SplyAdrBalUSD1K,SplyAdrBalUSD1M,SplyAdrTop100,SplyAdrTop10Pct,SplyAdrTop1Pct,SplyCur,SplyExpFut10yr,SplyFF,SplyMiner0HopAllNtv,SplyMiner0HopAllUSD,SplyMiner1HopAllNtv,SplyMiner1HopAllUSD,TxCnt,TxCntSec,TxTfrCnt,TxTfrValAdjNtv,TxTfrValAdjUSD,TxTfrValMeanNtv,TxTfrValMeanUSD,TxTfrValMedNtv,TxTfrValMedUSD,VelCur1yr,VtyDayRet180d,VtyDayRet30d,principal_market_price_usd,principal_market_usd,Volume,Annualized_DilutionRate,Total daily transaction volume,Velocity,The daily revenue of bitcoin miners,The accumulated revenue of bitcoin miners,P/E Ratio,Metcalfe,PM,<1d,1d ~ 1m,1m ~ 1q,1q ~ 6m,6m ~ 1y,1y ~ 2y,2y ~ 3y,3y ~ 4y,4y ~ 5y,5y~10y,>10y,One-year staking ratio,Two-year staking ratio,Three-year staking ratio,Four-year staking ratio,Five-year staking ratio,Ten-year staking ratio,Token Utility,PU Ratio,volatility30,volatility60,volatility90,volatility180,Token Utility_volatility,PU Ratio_volatility,state,cash,bitcoin,holding,quantiles_low,quantiles_high,total,roi_gross,roi_daily,sharpe
2022-05-27,"942,679.00","9,325.00","2,398,784.00","17,339,823.00",969.00,"481,619.00","7,463,892.00",44.00,"98,091.00","42,035,363.00","20,555,381.00","10,061,737.00","3,520,075.00","15,980.00",5.00,"147,205.00",94.00,"846,244.00","2,215.00",0.00,"14,681,544.00","299,677.00","25,275,367.00","1,670,874.00","5,838.00","35,181,497.00","5,744,667.00","71,918.00","1,653,715,161.00",138.00,"1,292,259.30","3,465,291.63","478,210,245.00","187,524,553,423.64",1.21,0.93,"545,088,669,358.95","545,069,029,219.99","418,510,426,944.33","451,648,033,359.06","29,897,409,688,833.63","29,897,409,688,833.63",0.00,0.00,1.83,0.00,0.70,17.60,"503,477.94","25,003.30","715,395,400.54","24,678.85","706,112,399.84","11,070.00","205,096,744.25","217,707,481.51",862.50,1.65,0.00,"24,677,885.63",862.50,"24,677,885.63",0.34,33.47,29.57,25.70,22.70,1.00,"28,612.04",-25.70,-27.04,"29,332.49",16.23,"27,352.05","29,332.49","43,308,902,974.30",0.00,0.00,0.12,0.00,880.10,"25,181,363.57",0.09,"16,547,933.81","4,544,280.35","468,436.77","6,554,043.15","10,523,746.11","1,862,841.85","11,801,644.21","13,134,748.75","14,602,135.95","778,825.21","3,264,130.01","17,275,631.66",34.40,"11,118,630.26","18,644,728.76","19,039,649.48","6,542,247.13","17,517,961.72","18,970,471.45","2,313,706.07","15,564,552.82","19,043,998.87","19,004,057.80","18,792,093.96","17,966,602.00","16,197,352.45","11,950,462.83","772,494.90","2,9

In [28]:
figure =PUR.figure_signal(data=data)

In [31]:
figure = PUR.figure_portfolio(data=data)

In [32]:
figure = PUR.figure_returns(data=data)